In [1]:
import torch
from torch import Tensor
import dlc_practical_prologue as prologue

In [394]:
class Args:
    full = True
    tiny = True
    seed = 42
    cifar = True
    data_dir = './data'
    file = None

args = Args()

In [5]:

# Now use args as usual
if args.seed >= 0:
    torch.manual_seed(args.seed)

train_input, train_target, test_input, test_target = prologue.load_data(
     one_hot_labels=False, normalize=True, flatten=True
)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [179]:
def nearest_classification(train_input, train_target, x):
    dist = (train_input - x).pow(2).sum(1).view(-1)
    _, n = torch.min(dist, 0)
    return train_target[n.item()]

In [231]:
def nearest_classification(train_input, train_target, x):
    ## Substracts
    __, index = torch.min(torch.sqrt(torch.sum(torch.pow((train_input - x), 2),1)),0)
    nearest_value = train_target[int(index)]
    return nearest_value
    

In [232]:
check = nearest_classification(train_input, train_target, test_input[455])

In [233]:
check

tensor(1)

In [234]:
test_target[455]

tensor(1)

In [84]:
sq = torch.pow(subs, 2)

In [88]:
summed = torch.sum(sq, 1)

In [95]:
sqrted = torch.sqrt(summed)

In [132]:
__, index = torch.min(sqrted, -1)
int(index)

360

In [164]:
int(torch.min(sqrted, -1)[1])

360

In [12]:
x_train = torch.full((4,5),10)
x_train

tensor([[10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10]])

In [51]:
x_test = torch.arange(1,6)
x_test

tensor([1, 2, 3, 4, 5])

In [55]:
subs = x_train - x_test
subs

tensor([[9, 8, 7, 6, 5],
        [9, 8, 7, 6, 5],
        [9, 8, 7, 6, 5],
        [9, 8, 7, 6, 5]])

In [53]:
squared = torch.pow(subs, 2)
squared

tensor([[81, 64, 49, 36, 25],
        [81, 64, 49, 36, 25],
        [81, 64, 49, 36, 25],
        [81, 64, 49, 36, 25]])

Sum accross column = per row

In [54]:
sum_per_row = torch.sum(squared, 1)
sum_per_row

tensor([255, 255, 255, 255])

In [57]:
torch.sqrt(squared)

tensor([[9., 8., 7., 6., 5.],
        [9., 8., 7., 6., 5.],
        [9., 8., 7., 6., 5.],
        [9., 8., 7., 6., 5.]])

### Transposed

In [47]:
sum_per_row.view(4,-1)

tensor([[320],
        [320],
        [320],
        [320]])

<h4><span style="color:green">  Corrections and improvements</span></h4>

<span style="color:green"> No need to wrap with torch.func if the object is already a torch object !</span>

### 2- Error estimation

In [253]:
def compute_nb_errors(train_input, train_target, test_input, test_target, mean = None, proj = None):
    if mean is not None:
        mean_t = mean.view(-1,1)
        train_minus_mean = train_input - mean_t
        test_minus_mean = test_input - mean_t
    else:
        train_minus_mean =  train_input - train_input[0].mean()
        test_minus_mean =  test_input - test_input[0].mean()
        
    if proj is not None:
        train_res_data = train_minus_mean.mm(proj.transpose(0,1))
        test_res_data = test_minus_mean.mm(proj.transpose(0,1))

    misclassified = 0
    for i, test_row in enumerate(test_res_data):
        label_test = nearest_classification(train_res_data, train_target, test_row)
        if str(test_target[i]) != str(label_test):
            misclassified += 1
    
    return misclassified 
    

In [354]:
def compute_nb_errors(train_input, train_target, test_input, test_target, mean = None, proj = None):
    if mean is not None:
        mean_t = mean.view(-1,1)
        train_input = train_input - mean_t
        test_input = test_input - mean_t
        
    if proj is not None:
        train_input = train_input.mm(proj.transpose(0,1))
        test_input = test_input.mm(proj.transpose(0,1))

    misclassified = 0
    for i, test_row in enumerate(test_input):
        label_test = nearest_classification(train_input, train_target, test_row)
        if str(test_target[i]) != str(label_test):
            misclassified += 1
    
    return misclassified 
    

<h3><span style="color:green">  Corrections and improvements</span></h4>

<span style="color:green"> Smarted way to access response label, better function to transpose </span>

In [355]:
def compute_nb_errors(train_input, train_target,
                      test_input, test_target,
                      mean = None, proj = None):

    if mean is not None:
        train_input = train_input - mean
        test_input = test_input - mean

    if proj is not None:
        train_input = train_input @ proj.t()
        test_input = test_input @ proj.t()

    nb_errors = 0

    # With loop, but I prefer clearer code when counting errors
    for n in range(test_input.size(0)):
        if test_target[n] != nearest_classification(train_input, train_target, test_input[n]):
            nb_errors = nb_errors + 1

    return nb_errors

#### Draft

In [255]:
proj = torch.randn(100, 784)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj)

In [258]:
misclassified / test_input.size()[0] * 100


18.7

In [ ]:
mean = train_input[0].mean()

In [216]:
train_minus_mean =  train_input - train_input[0].mean()
test_minus_mean =  test_input - test_input[0].mean()

In [218]:
proj = torch.randn(100, 784)

In [226]:
proj.transpose(0, 1)

torch.Size([784, 100])

In [248]:
train_input.size()

torch.Size([1000, 784])

In [228]:
train_res_data = train_minus_mean.mm(proj.transpose(0,1))
test_res_data = test_minus_mean.mm(proj.transpose(0,1))

In [237]:
train_res_data.size()

torch.Size([1000, 100])

In [241]:
test_res_data.size()

torch.Size([1000, 100])

In [246]:
nearest_classification(train_res_data, train_target, test_res_data[856])

tensor(5)

In [247]:
test_target[856]

tensor(5)

In [217]:
train_input.size()

torch.Size([1000, 784])

In [195]:
train_input.mean(1).size()

torch.Size([1000])

In [208]:
train_input[0][0] - train_input[0].mean()

tensor(-0.4512)

In [206]:
train_input[0].mean()

tensor(0.0309)

In [203]:
train_input - train_input.mean(1).view(1000,1)

tensor([[-0.4512, -0.4512, -0.4512,  ..., -0.4512, -0.4512, -0.4512],
        [-0.5097, -0.5097, -0.5097,  ..., -0.5097, -0.5097, -0.5097],
        [-0.3187, -0.3187, -0.3187,  ..., -0.3187, -0.3187, -0.3187],
        ...,
        [-0.5174, -0.5174, -0.5174,  ..., -0.5174, -0.5174, -0.5174],
        [-0.3716, -0.3716, -0.3716,  ..., -0.3716, -0.3716, -0.3716],
        [-0.4198, -0.4198, -0.4198,  ..., -0.4198, -0.4198, -0.4198]])

In [199]:
train_input.mean(1).size()

torch.Size([1000])

### 3 - PCA

In [369]:
def PCA(x):
    mean = x.mean(0)
    b = x - mean
    eigen_values, eigen_vectors = torch.linalg.eigh(b.t().mm(b))
    right_order = eigen_values.abs().argsort(descending = True)
    eigen_vectors = eigen_vectors.t()[right_order]
    return mean, eigen_vectors


# def PCA(x):
#     mean = x.mean(0)
#     b = x - mean
#     Sigma = b.t() @ b
#     eigen_values, eigen_vectors = Sigma.eig(True)
#     right_order = eigen_values[:, 0].abs().sort(descending = True)[1]
#     eigen_vectors = eigen_vectors.t()[right_order]
#     return mean, eigen_vectors

In [370]:
mean, eigen_vectors = PCA(train_input)

In [329]:
# eigen_values, eigen_vectors  = torch.linalg.eigh(x.mm(x.t()))
# eigen_values

In [328]:
# sorted_indices = torch.argsort(eigen_values, descending=True)
# sorted_indices

In [311]:
# right_order = eigen_values.abs().argsort(descending = True)
# right_order

In [330]:
# eigen_vectors = eigen_vectors.t()[right_order]
# eigen_vectors

In [327]:
# sorted_indices = torch.argsort(eigen_values2, descending=True)
# sorted_indices

In [326]:
# eigenvectors = torch.linalg.eig(x.mm(x.t()))[1]
# eigenvectors[0]

In [371]:
eigen_vectors

tensor([[-1.5607e-15,  1.9372e-07,  1.1921e-06,  ...,  2.8284e-08,
          2.8284e-08,  2.8284e-08],
        [ 1.8332e-16, -2.6077e-08, -7.4506e-08,  ..., -5.0282e-09,
         -5.0282e-09, -5.0282e-09],
        [-3.4568e-16, -9.3132e-09,  1.1176e-07,  ...,  1.2443e-08,
          1.2443e-08,  1.2443e-08],
        ...,
        [ 0.0000e+00, -2.9116e-04, -6.0156e-05,  ..., -1.6670e-02,
          1.1800e-01,  1.1800e-01],
        [ 1.0000e+00,  5.8403e-11,  5.8193e-11,  ...,  5.8381e-11,
          5.8381e-11,  5.8381e-11],
        [ 0.0000e+00,  6.0111e-11, -9.2610e-11,  ...,  3.7327e-08,
         -7.0711e-01,  7.0711e-01]])

In [366]:
eigen_vectors.size()

torch.Size([784, 784])

In [273]:
x.mean(0).size()

torch.Size([784])

In [272]:
x.size()

torch.Size([1000, 784])

<h4><span style="color:green">  Corrections and improvements</span></h4>

In [322]:
def PCA(x):
    mean = x.mean(0)
    b = x - mean
    Sigma = b.t() @ b
    eigen_values, eigen_vectors = Sigma.eig(True)
    right_order = eigen_values[:, 0].abs().sort(descending = True)[1]
    eigen_vectors = eigen_vectors.t()[right_order]
    return mean, eigen_vectors


In [323]:
PCA(x)

IndexError: too many indices for tensor of dimension 1

<span style="color:green"> The problem is that .eig has been depreciated.</span>
<br>
<span style="color:green"> Another very important thing I had to correct was how to create the covariance matrice so it is of the dimension of the features and not of the data... !</span>

### 4 - Check that all this makes sense


Random projection

In [335]:
proj = torch.randn(100, 784)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj)
misclassified / test_input.size(1) * 100

24.872448979591837

In [374]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj)
misclassified / test_input.size(1) * 100

21.93877551020408

In [380]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:3,:])
misclassified / test_input.size(1) * 100

76.14795918367348

In [382]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:10,:])
misclassified / test_input.size(1) * 100

27.29591836734694

In [383]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:10,:])
misclassified / test_input.size(1) * 100

27.29591836734694

In [384]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:50,:])
misclassified / test_input.size(1) * 100

19.77040816326531

In [385]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:100,:])
misclassified / test_input.size(1) * 100

20.918367346938776

In [361]:
train_target.size()

torch.Size([1000])

In [395]:
train_input, train_target, test_input, test_target = prologue.load_data(cifar=True,
     one_hot_labels=False, normalize=True, flatten=True
)

* Using CIFAR
Files already downloaded and verified
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [388]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:3,:])
misclassified / test_input.size(1) * 100

27.311197916666668

In [389]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:10,:])
misclassified / test_input.size(1) * 100

24.837239583333336

In [390]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:10,:])
misclassified / test_input.size(1) * 100

24.837239583333336

In [391]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:50,:])
misclassified / test_input.size(1) * 100

23.990885416666664

In [392]:
mean, proj = PCA(train_input)
misclassified = compute_nb_errors(train_input =  train_input, train_target = train_target, test_input = test_input, test_target = test_target, mean = None, proj = proj[:100,:])
misclassified / test_input.size(1) * 100

24.251302083333336

In [393]:
train_target.size()

torch.Size([1000])

It makes sense that since CIFAR has a more complex distribution the algorithms work less well. It also makes sense that reducing too much the data leads to a perf degradation but finding the optimal dimension reduction leads to an improvement in performance.